In [1]:
import cv2

img = cv2.imread("../input/beauty-detection-data-set/images to predict/3.jpg")

img.shape

In [2]:
import matplotlib.pyplot as plt

plt.imshow(img)


## **Image Processing**

In [3]:
train_dir = "../input/beauty-detection-data-set/train"
test_dir = "../input/beauty-detection-data-set/test"
val_dir = "../input/beauty-detection-data-set/valid"

In [4]:
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                                                                                   
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)


In [5]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           class_mode = "binary"
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (224,224),
                                         class_mode = "binary"
                                         )
val_data = val_gen.flow_from_directory(val_dir,
                                       target_size = (224,224),
                                       class_mode = "binary"
                                       )

In [6]:
labels = list(train_data.class_indices.keys())

labels

In [7]:
# visualizing the train data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = train_data.next()

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(labels[int(label[i])])
  plt.axis("off")


In [8]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
                             layers.Conv2D(filters= 62, kernel_size= 2, activation="relu", input_shape=(224,224,3)),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 62, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 62, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Flatten(),


                             layers.Dense(128, activation= "relu"),
                             layers.Dropout(0.5),

                             layers.Dense(1, activation= "sigmoid")
])

In [9]:
model.summary()

In [10]:
# compiling the model

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [11]:
# fitting the model

model.fit(train_data,
          epochs = 10,
          steps_per_epoch = len(train_data),
          validation_data = val_data,
          validation_steps = len(val_data)
          )

In [12]:
model_evaluation = model.evaluate(test_data)

In [13]:
print(f"Model Accuracy: {model_evaluation[1] * 100 : 0.2f} %")

In [14]:
# visualizing the test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):

  image, label = test_data.next()

  model_pred = model.predict(image) 

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction : {labels[int(tf.round(model_pred[i]))]}\n Original: {labels[int(label[i])]}")
  plt.subplots_adjust(top= 1.25)
  plt.axis("off")
